# Simple PDB Check
### find component details

In [1]:
#!/usr/bin/python3
### get general libraries
import os
import sys
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interactive_output, interact

from notebook.services.config import ConfigManager

In [2]:
### get PDB credentials
cwd = os.getcwd()
# check where this is:
#cwd
sys.path.insert(1, cwd+"/Notebooks")
import setEnv
setEnv.SetEnv()

# import API and check authentication
import itkdb
myClient = itkdb.Client()
myClient.user.authenticate()
user=myClient.get('getUser', json={'userIdentity': myClient.user.identity})
print(user['firstName'])

Done
Kenneth


## Select institution and project

In [3]:
# institute widget
selInst = widgets.Dropdown(
    options=['LIV', 'GL'],
    value='GL',
    description='Institution:',
    disabled=False,
    layout = widgets.Layout(width='50%', height='80px')
)

# project widget
selProj = widgets.Dropdown(
    options=['S', 'P'],
    value='S',
    description='Project:',
    disabled=False,
    layout = widgets.Layout(width='50%', height='80px')
)

#opts = widgets.HBox([selInst, selProj])

In [4]:
compMap=[]
def Get_df(myInst, myProj):
    global compMap
    compList=list(myClient.get('listComponents',json={'institution':myInst, 'project':myProj }))
    compMap=[{"code":w['code'], "ASN":w['serialNumber'], "altID":w['alternativeIdentifier'], "typeCode":w['componentType']['code'],"projCode":w['project']['code'],"instCode":w['institution']['code'],"locCode":w['currentLocation']['code'],"state":w['state']} for w in compList]
    #print("length:",len(compMap))
    df_comp=pd.DataFrame(compMap)
    display(df_comp.drop(["code","ASN","state"], axis=1))
    return

out_df=interactive_output(Get_df, {'myInst':selInst, 'myProj':selProj} )

In [5]:
display( widgets.VBox( [selInst, selProj, out_df]) )

## Select componentType and alternativeID
### (from selected institution and project)

In [8]:
# component type selection based on above inst & proj selections
def set_compType(selCompType):
    global compMap

def set_compType_options(_):
    global compMap
    selCompType.options = list( dict.fromkeys( [x['typeCode'] for x in compMap] ) )

# component widget
selCompType = widgets.Dropdown(
    options=[],
    #value='S',
    description='Select Component Type:',
    disabled=False,
    layout = widgets.Layout(width='50%', height='80px')
)

selInst.observe(set_compType_options)
selProj.observe(set_compType_options)

#interact function - commented as later interact call duplicates
#interact(set_compType, selCompType = selCompType)

In [12]:
# alterative ID selection based on above inst & proj & compType selections
def set_altID(selCompType, selAltID):
    global compMap

def set_altID_options(_):
    global compMap
    selAltID.options = [x['altID'] for x in compMap if x['typeCode']==selCompType.value] 

# component widget
selAltID = widgets.Dropdown(
    options=[],
    #value='S',
    description='Select Component altID:',
    disabled=False,
    layout = widgets.Layout(width='50%', height='80px')
)

selCompType.observe(set_altID_options)

In [19]:
#interact function - gives both compType and altID selection
interact(set_altID, selCompType = selCompType, selAltID = selAltID)

interactive(children=(Dropdown(description='Select Component Type:', layout=Layout(height='80px', width='50%')…

<function __main__.set_altID(selCompType, selAltID)>

In [16]:
def Get_comp(altID):
    global compMap
    df_comp=pd.DataFrame(compMap)
    try:
        display(df_comp.query('altID=="'+altID+'"'))
    except:
        print("poorly defined identifier")
    return

In [17]:
out_comp=interactive_output(Get_comp, {'altID':selAltID} )

In [18]:
display( widgets.VBox([out_comp]) )